<a href="https://colab.research.google.com/github/AxithChoudhary/covid-19-chest-image-prediction/blob/main/Covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
print(tf.__version__)
!python --version

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Study material /ML/NEPTL/Time series /Covid dataset/15.1 covid19.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [ ]:
test_path="/tmp/covid19/test"
train_path="/tmp/covid19/train"

In [ ]:
batch_size=32
img_height=img_width=100

In [ ]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(train_path,image_size=(img_width,img_height),label_mode='int',batch_size=batch_size)

In [ ]:
val_ds=tf.keras.preprocessing.image_dataset_from_directory(test_path,image_size=(img_width,img_height),label_mode='int',batch_size=batch_size)

In [ ]:
train_ds.prefetch(1)

In [ ]:
class_names=train_ds.class_names
num_classes=len(class_names)
print(class_names)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))
for img ,label in val_ds.take(1):
  for i in range(12):
    ax=plt.subplot(4,3,i+1)
    plt.imshow(img[i].numpy().astype("uint8"))
    plt.title(class_names[label[i]])
    plt.axis("off")

In [ ]:
norm_layer=tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
zoom_layer=tf.keras.layers.experimental.preprocessing.RandomZoom(height_factor=(0.1,0.3),width_factor=(0.1,0.3))
rot_layer=tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.2)

In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Input,Flatten,Dense,BatchNormalization,Dropout
from tensorflow.keras.models import Sequential

In [ ]:
def  get_model(num_classes):
  model=Sequential([
                    norm_layer,zoom_layer,rot_layer,
                    Conv2D(16,(5,5),padding='same',activation="relu"),
                    MaxPooling2D((2,2),strides=(1,1)),
                    BatchNormalization(),
                    Dropout(0.3),
                    Conv2D(16,(3,3),padding='same',activation="relu"),
                    MaxPooling2D((2,2),strides=(2,2)),
                    Dropout(0.3),
                    Flatten(),
                    Dense(256,activation='relu'),
                    Dropout(0.5),
                    Dense(num_classes,activation='softmax')

  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False,reduction=tf.keras.losses.Reduction.AUTO),metrics=['accuracy'])
  return model

In [ ]:
model=get_model(num_classes=num_classes)

In [ ]:
from traitlets.traitlets import validate
history=model.fit(train_ds,validation_data=val_ds,epochs=50)

In [ ]:
fig=plt.figure(figsize=(12,5))
fig.add_subplot(121)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('loss vs epoches')
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(['Traning','Validation'],loc='best')

fig.add_subplot(122)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('accuracy vs epochs')
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(['Traning','Validation'],loc='best')

plt.show()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_datagen=ImageDataGenerator()

test_ds=test_datagen.flow_from_directory(test_path,target_size=(img_height,img_width),batch_size=1,class_mode='categorical',color_mode='rgb',shuffle=False,seed=2021)


In [ ]:
test_ds.reset()
predictions=model.predict(test_ds,steps=len(test_ds.filenames),verbose=1)

In [ ]:
import pandas as pd

predicted_class_indices=np.argmax(predictions,axis=1)
labels=(test_ds.class_indices)
labels=dict((v,k) for k,v in labels.items())
pred_labels=[labels[k] for k in predicted_class_indices]
filemanes=test_ds.filenames
results=pd.DataFrame({"Filename":test_ds.filenames,"Predictions":pred_labels})
results.tail(60)